# STROKE DECISION SUPPORT SYSTEM ICA 2

In [1]:
#Importation of Required Library
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix, classification_report



#This importation is to ignore warnings
import warnings
warnings.filterwarnings("ignore")



In [2]:
#Importation and Loading of Datasets
df = pd.read_csv('healthcare-dataset-stroke-data.csv', index_col=0)
df.head()

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
id,,,,,,,,,,,
9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
51676,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
60182,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1


# STATISTICAL ANALYSIS

In [3]:
#Shape of the Datsaset Showing the Number of Rows and Columns
df.shape

(5110, 11)

In [5]:
#Checking the number of categorical columns in the dataset
#We have five categorical column in the dataset 
cat = df.select_dtypes(include='object').columns
print ("There are " ,len(cat), "categorical column in the dataset :\n", cat)

There are  5 categorical column in the dataset :
 Index(['gender', 'ever_married', 'work_type', 'Residence_type',
       'smoking_status'],
      dtype='object')


In [7]:
#Checking the number of Numberical columns in the dataset
numer = df.select_dtypes(include=np.number).columns
print ("There are",len(numer), "Numerical column in the dataset which are the independent variables:\n", numer)

There are 6 Numerical column in the dataset which are the independent variables:
 Index(['age', 'hypertension', 'heart_disease', 'avg_glucose_level', 'bmi',
       'stroke'],
      dtype='object')


In [8]:
#details of the last 5 rows of the dataset  
df.tail()

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
id,,,,,,,,,,,
18234,Female,80.0,1,0,Yes,Private,Urban,83.75,NaN,never smoked,0
44873,Female,81.0,0,0,Yes,Self-employed,Urban,125.20,40.0,never smoked,0
19723,Female,35.0,0,0,Yes,Self-employed,Rural,82.99,30.6,never smoked,0
37544,Male,51.0,0,0,Yes,Private,Rural,166.29,25.6,formerly smoked,0
44679,Female,44.0,0,0,Yes,Govt_job,Urban,85.28,26.2,Unknown,0


In [14]:
#Checking for Duplicate values
#Checking for Duplicate values and missing data
dup_count = df.duplicated().sum()
print(f"Duplicate rows: {dup_count}")

# Missing values per column (count and percent)
missing_count = df.isnull().sum()
missing_pct = (df.isnull().mean() * 100).round(2)
missing_summary = pd.concat([missing_count, missing_pct], axis=1)
missing_summary.columns = ['missing_count', 'missing_pct']
print("\nMissing values per column:\n", missing_summary)

# Columns that have missing values
missing_cols = missing_count[missing_count > 0].index.tolist()
print("\nColumns with missing values:", missing_cols)

# Rows that contain any missing values (count and preview)
rows_with_missing = df[df.isnull().any(axis=1)]
print("\nNumber of rows with at least one missing value:", rows_with_missing.shape[0])
rows_with_missing.head()

Duplicate rows: 0

Missing values per column:
                    missing_count  missing_pct
gender                         0         0.00
age                            0         0.00
hypertension                   0         0.00
heart_disease                  0         0.00
ever_married                   0         0.00
work_type                      0         0.00
Residence_type                 0         0.00
avg_glucose_level              0         0.00
bmi                          201         3.93
smoking_status                 0         0.00
stroke                         0         0.00

Columns with missing values: ['bmi']

Number of rows with at least one missing value: 201


,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
id,,,,,,,,,,,
51676,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
27419,Female,59.0,0,0,Yes,Private,Rural,76.15,NaN,Unknown,1
8213,Male,78.0,0,1,Yes,Private,Urban,219.84,NaN,Unknown,1
25226,Male,57.0,0,1,No,Govt_job,Urban,217.08,NaN,Unknown,1
61843,Male,58.0,0,0,Yes,Private,Rural,189.84,NaN,Unknown,1
